# 字符串的定义

* 字符集：有穷的一组字符构成的带有序关系的集合；
* 字符串：就是字符集里有限个元素的排列，可以看做一类特殊的线性表。

# 字符串的相关概念

1. 字符串的长度：字符串中字符的个数，空字符串的长度为0。下面用 |s|表示字符串s的长度；

2. 字符在字符串的位置：字符串中字符顺序排列，从0开始计数，每个字符都有其确定位置或下标；

3. 字符串相等：两个字符串长度相等且相应位置的字符相等；

4. 字典序：字符串之间的一种序关系，简单说就是从左到右开始第一个不相等的字符决定了2个字符串的大小；如果都一致，则长度较短的字符串相对小；

5. 字符串拼接：s1 + s2

6. 子串关系： 称字符串 s1 是字符串 s2 的子串，如果满足： $s_2 = s + s_1 + s^{*}$;即 s1 和 s2 中的一个连续片段一致；
    * 空字符串和本身都是字符串的子串；
    * 子串可以多次出现，且下标可能会重叠，如 babb 相对于 babbabbbbabb.
    
7. 前缀和后缀：
    * 前缀： $s_2 = s_1 + s^{*}$;
    * 后缀： $s_2 = s^{*} + s_1$;
   
8. 串替换： 将字符串中明确下标范围的一段子串进行替换

# 字符串匹配

假设有两个字符串：
    $$t=t_1t_2t_3\dots t_{n-1}$$
    $$p=p_1p_2p_3\dots p_{m-1}$$
字符串匹配就是在字符串t中查找和p相同字符串的过程，下面称t为**目标串**，p为**模式串**，一般情况下 m<<n.

字符串比较的基础是字符比较。

* 如果从字符i开始，目标串的每一个字符都和模式串里的字符一致，则找到了一个匹配的模式串；

* 如果比较过程中，遇到了一对不同的字符，说明目标串里从字符i开始的子串和模式串不匹配。串匹配算法的关键在两点：
    1. 怎样选择开始比较的字符对；
    2. 遇到不匹配的字符对时下一步怎么办
    
    对这两点的不同处理策略，形成了不同的字符串匹配算法。下面介绍朴素匹配算法和KMP匹配算法。

## 朴素匹配算法

最简单的朴素匹配算法，对以上2点的处理策略是：
1. 从左往右依次比较；
2. 发现不匹配，从**当初开始比较位置的下一位置**重新开始匹配

In [9]:
# 只匹配1次
def naive_matching(t,p):
    m,n = len(p),len(t)
    i,j = 0,0
    
    while i<m and j<n:
        if p[i] == t[j]: 
            i,j = i+1, j+1 # 字符匹配就一起前进 
        else:
            i,j = 0, j-(i-1) #如果不匹配，模式串从0开始，目标串从上次比较位置的下一个位置开始 j -(i-1)
    if i == m:
        return j-i
    else:
        return -1

In [10]:
naive_matching(t='assmddmsmcd',p='smc')

7

上面的朴素匹配算法效率很低，原因在于当发现不匹配时，模式串和目标串都要进行回溯。算法的复杂度是：$O(m\times n)$。最糟糕的案例是：
1. t = '00000000000000000000001'
2. p = '001'

效率低的原因在于：
1. 把每次的字符匹配当成完全独立的操作，没有利用字符是有穷的特点，把字符当做随机出现的量；

2. 也没有利用前面匹配过程中的有效信息。

其他改进的算法都是在这个基础上利用字符的特点进行优化的。

## KMP算法(无回溯串匹配算法)

介绍参考：
1. http://www.ruanyifeng.com/blog/2013/05/Knuth%E2%80%93Morris%E2%80%93Pratt_algorithm.html
2. https://www.pythonf.cn/read/107379
3. https://leetcode-cn.com/problems/implement-strstr/solution/kmp-suan-fa-xiang-jie-by-labuladong/

In [246]:
def pmt_table(s_str):
    # s模式串
    m,s,pmt = 0,-1,[-1]*len(s_str)

    while m<len(s_str)-1:
        if s == -1 or s_str[m] == s_str[s]:
            m += 1
            s += 1
            pmt[m] = s 
        else:
            s = pmt[s]
    return pmt


def kmp(m,s):
    # m主串，s模式串
    i,j = 0,0 
    pmt = pmt_table(s)
    while i<len(m) and j<len(s):
        # 模式串循环结束，匹配成功；否则失败
        if m[i] == s[j] or j== -1:
            i,j = i+1,j+1
        else:
            j = pmt[j]
    if j == len(s):
        return i-j
    return -1

In [120]:
kmp('ababababc','ababc')

4

In [247]:
kmp('ababc','ab')

0

kmp算法的复杂度是：$O(m + n)$

# 常见技巧

## 状态自动机

当处理一个正在不断输入过程中的字符串，要根据 程序目前的状态 $s^0$ 以及根据当前输入的字符 c 来决定程序的下一个状态 $s^1$。
一般状态繁多，如果单纯通过 'if...else...'来写，会出现比较臃肿的代码，并且很可能还会遗漏一些状态。

一种比较好的处理方式是，列出全部的 $s^0$ - c - $s^1$。下面通过一个例子来说明。

题目：[字符串转换整数 (atoi)](https://leetcode-cn.com/problems/string-to-integer-atoi/)

分析如下：
1. 4种状态：start/end(开始/结束)、signed(当前输入字符是符号)、in_number(当前输入字符是数字)
2. 将输入的字符分成4类：空格(' ')、符号( +/-)、数字(number)、其他(others)
3. 从4种状态经过输入的4类字符，可以得到16个下一状态（都包含在4种状态之中）。我们一一列出如下：

|状态\字符|' '|+/-|number|others|
|:--|:--|:--|:--|:--|
|start|start|signed|in_number|end|
|signed|end|end|in_number|end|
|in_number|end|end|in_number|end|
|end|end|end|end|end|

我们只需要通过代码把这个过程实现即可

In [677]:
INT_MAX = 2 ** 31 - 1
INT_MIN = -2 ** 31

class Automachine():
    def __init__(self):
        self.state = 'start'
        self.sign = 1
        self.ans = 0
        self.table = {'start':['start','signed','in_number','end'],
                     'signed':['end','end','in_number','end'],
                     'in_number':['end','end','in_number','end'],
                     'end':['end','end','end','end']}
    
    def get_colSeq(self,c):
        if c.isspace():
            return 0
        elif c == '-' or c=='+':
            return 1
        elif c.isdigit():
            return 2
        return 3
    
    def get(self,c):
        self.state = self.table[self.state][self.get_colSeq(c)]
        if self.state == 'in_number':
            self.ans = self.ans*10 + int(c)
            self.ans = min(self.ans,INT_MAX) if self.sign == 1 else min(self.ans,-INT_MIN)
        elif self.state == 'signed':
            self.sign = 1 if c == '+' else -1 
    
class Solution():
    def myAtoi(self, s: str) -> int:
        automachine = Automachine()
        for c in s:
            automachine.get(c)
        
        return automachine.ans*automachine.sign

In [678]:
s='-5-'
t=Solution()
t.myAtoi(s)

-5

# 字符串练习

## [罗马数字转整数](https://leetcode-cn.com/problems/roman-to-integer/submissions/)

In [11]:
class Solution:
    def romanToInt(self, s: str) -> int:
        map_dict={'I':1,'V':5,'X':10,'L':50,'C':100,'D':500,'M':1000,'IV':4,'IX':9,'XL':40,'XC':90,'CD':400,'CM':900}
        ten_binary = 0
        i = 0
        while i < len(s):
            char = s[i]
            slice_char = s[i:(i+2)]  
            if slice_char in map_dict:
                ten_binary += map_dict[slice_char]
                i += 2
            else:
                ten_binary += map_dict[char]
                i += 1
        return ten_binary

In [12]:
s="IX"
a=Solution()
a.romanToInt(s)

9

## [最长公共前缀](https://leetcode-cn.com/problems/longest-common-prefix/submissions/)

In [39]:
class Solution:
    def longestCommonPrefix(self, strs: list) -> str:
        if len(strs) == 0:
            return ''
        prefix,count = strs[0],len(strs)
        
        for i in range(1,count):
            prefix = self.twolcp(prefix,strs[i])
            if prefix == '':
                break 
        return prefix
        
        
    def twolcp(self,str1,str2):
        length,index = min(len(str1),len(str2)),0
        
        while index<length and str1[index] == str2[index]:
            index +=1
        return str1[:index]
    
    def longestCommonPrefix_v2(self, strs: list) -> str:
        if len(strs) == 0:
            return ''
        if len(strs) == 1:
            return strs[0]
        
        s1,strs_1,common_prefix = strs[0],strs[1:],''
        
        bool_v = 1
        for i in range(len(s1)):
            s = s1[i]
            for j in strs_1:
                if len(j)<i+1 or s!=j[i]:
                    bool_v = 0
                    break 
            
            if bool_v:
                common_prefix += s1[i]
            else:
                break
        return common_prefix

In [38]:
strs = ["flower","flow","flight"]
# strs=['aa','a']
a=Solution()
a.longestCommonPrefix(strs)


'fl'

## [有效的括号](https://leetcode-cn.com/problems/valid-parentheses/submissions/)

In [78]:
class Solution:
    def isValid(self, s: str) -> bool:
        if len(s)%2 == 1:
            return False
        
        maps = {')':'(',"}":"{","]":"["}
        left_l = []
        for i in s:
            if i in maps:
                if not left_l or left_l[-1] != maps[i]:
                    return False
                left_l.pop()
            else:
                left_l.append(i)
   
        return len(left_l) == 0
            
                
        
    def isValid_v2(self, s: str) -> bool:
        if len(s)%2 == 1:
            return False
        
        maps = {'(':1,')':-1,'{':2,'}':-2,'[':3,']':-3}
        map_lst = [maps[i] for i in s]
        left_l = []
        for i in map_lst:
            #入栈
            if i>0:
                left_l.append(i)
            else:
                if len(left_l) == 0:
                    return False
                elif i + left_l[-1] != 0:
                    return False
                else:
                    left_l.pop(-1)
        if len(left_l) == 0:
            return True
        else:
            return False
                

In [79]:
s="{}{{[]}}"
a=Solution()
a.isValid(s)

True

In [ ]:
class Solution:
    def checkValidString(self, s: str) -> bool:
        maps = {')':'('}
        left = []
        right = []
        :
            if i in maps:
                if not left_l or left_l[-1] != maps[i]:
                    return False
                left_l.pop()
            else:
                left_l.append(i)
   
        return len(left_l) == 0
        
        

## [机器人能否返回原点](https://leetcode-cn.com/problems/robot-return-to-origin/)

In [432]:
class Solution:
    def judgeCircle(self, moves: str) -> bool:
        x,y = 0,0
        for i in moves:
            if i == 'U':
                x += 1
            elif i == 'D':
                x -= 1
            elif i == 'L':
                y += 1
            elif i == 'R':
                y -= 1
        return x == y  == 0
        
    def judgeCircle_v2(self, moves: str) -> bool:
        move_dict = {}
        for i in moves:
            move_dict[i] = move_dict.get(i,0) + 1
            if move_dict[i]>len(moves)/2:
                return False 
        return move_dict.get('U') == move_dict.get('D') and move_dict.get('L') == move_dict.get('R')

In [433]:
s=['UD','LL']
a=Solution()
for i in s:
    print(i,a.judgeCircle(i))

UD True
LL False


## [判断路径是否相交](https://leetcode-cn.com/problems/path-crossing/submissions/)

In [158]:
class Solution:
    def isPathCrossing(self, path: str) -> bool:
        x,y,j,res = 0,0,0,{(0,0)}
        while j<len(path):
            i = path[j]
            if i == 'N':
                x += 1
            elif i == 'S':
                x -= 1
            elif i == 'E':
                y += 1
            elif i == 'W':
                y -= 1
    
            if (x,y) not in res:
                res.add((x,y))
            else:
                return True 

            j += 1
        return False

In [159]:
t=Solution()
t.isPathCrossing("NESWW")

True

## [实现 strStr()](https://leetcode-cn.com/problems/implement-strstr/)

In [121]:
class Solution:
   
    def strStr(self, haystack: str, needle: str) -> int:
        if needle == '':
            return 0
        
        pmt_table = self.pmt(needle)
        i,j = 0,0
        while i<len(haystack) and j<len(needle):
            if j == -1 or haystack[i] == needle[j]:
                i,j = i+1, j+1
            else:
                j = pmt_table[j]
        if j == len(needle):
            return i - j 
       
        return -1
    
    
    def pmt(self,strs):
        pmt_table = [-1]*len(strs)
        m,s = 0,-1
        
        while m<len(strs)-1:
            if s ==  -1 or strs[m] == strs[s]:
                m,s = m+1, s+1
                pmt_table[m] = s
            else:
                s = pmt_table[s]
        return pmt_table

In [122]:
haystack = "hello"
needle = "ll"
a=Solution()
a.strStr(haystack,needle)

2

## [外观数列](https://leetcode-cn.com/problems/count-and-say/)

In [141]:
class Solution:
    def countAndSay(self, n: int) -> str:
        for i in range(1,n+1):
            if i == 1:
                 re = '1'
            else:
                
                a,re  = self.conti_str(re,'')
        return re
    
    def conti_str(self,s,state_s):
        length = len(s)
        if length == 0:
            return '',state_s
        i = 0
        cnt = 1
        while i<length-1 and s[i+1] == s[i]:
                cnt += 1
                i += 1
        state_s +=  str(cnt)+ s[i]
        return self.conti_str(s[(i+1):],state_s)

In [143]:
a=Solution()
a.countAndSay(4)

'1211'

## [最后一个单词的长度](https://leetcode-cn.com/problems/length-of-last-word/)

In [177]:
class Solution:
    def lengthOfLastWord(self, s: str) -> int:
               
        i,empty_bool, res =0,1,0
        while i <len(s):
            char = s[i]
            if char != ' ':
                if empty_bool == 1:
                    res += 1
                else:
                    res = 1
                    empty_bool = 1
            else:
                empty_bool = 0
            i += 1
        return res

In [178]:
s="Hello World"
a=Solution()
a.lengthOfLastWord(s)

5

## [二进制求和](https://leetcode-cn.com/problems/add-binary/)

In [191]:
class Solution:
    def addBinary(self, a: str, b: str) -> str:
        a1,b1 = a[::-1],b[::-1]
        
        i,j,last_div,res = 0,0,0,''
        while i<len(a1) or j<len(b1):
            
            x =int(a1[i])  if i<len(a1) else 0
            y =int(b1[i])  if i<len(b1) else 0
            xy_sum = x + y + last_div
            last_div = xy_sum//2
            res += str(xy_sum%2)
            
            i += 1
            j += 1
        if last_div == 1:
            res += str(last_div)
        return res[::-1] 

In [193]:
a = "11"
b = "1"
t=Solution()
t.addBinary(a,b)

'100'

## [字符串相加](https://leetcode-cn.com/problems/add-strings/submissions/)

## [验证回文串](https://leetcode-cn.com/problems/valid-palindrome/)

In [217]:
class Solution:
    def isPalindrome(self, s: str) -> bool:
        import re 
        s1=''.join([i.lower() for i in re.findall('[a-zA-Z0-9]', s)])
        length = len(s1)
        first_part = s1[:length//2]
        
        if length%2 == 0:
            second_part = s1[length//2:]
        else:
            second_part = s1[(length//2+1):]
        return first_part == second_part[::-1]

In [222]:
s="A man, a plan, a canal: Panama"
t=Solution()
t.isPalindrome(s)

True

## [验证回文字符串 Ⅱ](https://leetcode-cn.com/problems/valid-palindrome-ii/submissions/)

In [471]:
class Solution:
    def validPalindrome(self, s: str) -> bool:
        left,right = 0,len(s)-1
        while left<right:
            if s[left] == s[right]:
                left += 1
                right -= 1
            else:              
                return self.checkPalindrome(s,left+1,right) or self.checkPalindrome(s,left,right-1)
        else:
            return True
        
    def checkPalindrome(self,s,left,right):
        while left<right:
            if s[left] != s[right]:
                return False
            left += 1
            right -= 1
        return True

In [472]:
s=['aba']
t=Solution()
for i in s:
    print(i,t.validPalindrome(i))

aba True


## [反转字符串](https://leetcode-cn.com/problems/reverse-string/submissions/)

In [227]:
class Solution:
    def reverseString(self, s: list) -> None:
        """
        Do not return anything, modify s in-place instead.
        """
        left,right = 0,len(s)-1
        while left<right:
            s[right],s[left]=s[left],s[right]
            left += 1
            right -= 1
        return s

In [229]:
s=["h","e","l","l","o"]
t=Solution()
t.reverseString(s)

['o', 'l', 'l', 'e', 'h']

## [仅仅反转字母](https://leetcode-cn.com/problems/reverse-only-letters/submissions/)

In [701]:
class Solution:
    def reverseOnlyLetters(self, S: str) -> str:
        l,r = 0,len(S)-1
        slst = list(S)
        while l<r:
            while l<r and not S[l].isalpha():
                l += 1
            while l<r and not S[r].isalpha():
                r -= 1 
            slst[r],slst[l] = slst[l],slst[r]
            l += 1
            r -= 1
        return ''.join(slst) 

In [702]:
S="7_28]"
t=Solution()
t.reverseOnlyLetters(S)
# 输出："j-Ih-gfE-dCba"


'7_28]'

## [反转字符串中的元音字母](https://leetcode-cn.com/problems/reverse-vowels-of-a-string/)

In [240]:
class Solution:
    def reverseVowels(self, s: str) -> str:
        vowel = 'aeiouAEIOU'
        s1 = list(s)
        left,right = 0,len(s)-1
        while left<right:
            while left<right and s[left] not in vowel:
                left += 1
            while left<right and s[right] not in vowel:
                right -= 1
                         
            s1[right],s1[left]=s1[left],s1[right]
            left += 1
            right -= 1
        return ''.join(s1)

In [241]:
s='hello'
t=Solution()
t.reverseVowels(s)

'holle'

## [赎金信](https://leetcode-cn.com/problems/ransom-note/)

In [248]:
class Solution:
    def canConstruct(self, ransomNote: str, magazine: str) -> bool:
        mlist = list(magazine)
        i = 0
        while i< len(ransomNote) and ransomNote[i] in mlist:
                mlist.remove(ransomNote[i])
                i += 1
        if i == len(ransomNote):
            return True
        return False

In [250]:
ransomNote = 'aa'
magazine = 'aba'
t=Solution()
t.canConstruct(ransomNote, magazine)

True

## [字符串中的第一个唯一字符](https://leetcode-cn.com/problems/first-unique-character-in-a-string/)

In [268]:
class Solution:
    def firstUniqChar(self, s: str) -> int:
        if len(s) == 0:
            return -1
        slist = list(s)
        unitstr = self.find_str(slist)
        if unitstr == '':
            return -1
        for i in range(len(s)):
            if s[i] == unitstr:
                return i
        
    def find_str(self,slist):
        if len(slist) == 0:
            return ''
        
        char = slist[0]
        slist = slist[1:]
        if char not in slist:
            return char 
        else:
            slist = [i for i in slist if i != char]
            return self.find_str(slist)

In [269]:
s = "aadadaad"
t=Solution()
t.firstUniqChar(s)

-1

## [字符串中的单词数](https://leetcode-cn.com/problems/number-of-segments-in-a-string/submissions/)

In [277]:
class Solution:
    def countSegments(self, s: str) -> int:
               
        i,char_bool, index =0,0,0
        while i <len(s):
            char = s[i]
            if char !=' ':
                if char_bool == 0:
                    index += 1
                    char_bool = 1
            else:
                char_bool = 0
            i += 1
        return index

In [278]:
s = "Hello, my name is John"
t=Solution()
t.countSegments(s)

5

## [反转字符串中的单词 III](https://leetcode-cn.com/problems/reverse-words-in-a-string-iii/)

In [602]:
class Solution:
    def reverseWords(self, s: str) -> str:
               
        i, unit_word,res =0,'',''
        while i <len(s):
            char = s[i]
            if char !=' ':
                unit_word += char
            else:
                res += unit_word[::-1] + char
                unit_word = ''
    
            i += 1
        res += unit_word[::-1]
        return res

In [426]:
s = "Let's take LeetCode contest"
t=Solution()
t.reverseWords(s)

"s'teL ekat edoCteeL tsetnoc"

## [重复的子字符串](https://leetcode-cn.com/problems/repeated-substring-pattern/submissions/)

In [373]:
class Solution:
    def repeatedSubstringPattern(self, s: str) -> bool:
        if len(s)<=1:
            return False
       
        t = s + s
        t1 = t[1:] + t[0]
        m = self.kmp(t1,s)
#         print(t1,s,m)
        return m>=0 and m<len(s)-1
            
    def pmt_table(self,s_str):
        # s模式串
        m,s,pmt = 0,-1,[-1]*len(s_str)

        while m<len(s_str)-1:
            if s == -1 or s_str[m] == s_str[s]:
                m += 1
                s += 1
                pmt[m] = s 
            else:
                s = pmt[s]
        return pmt


    def kmp(self,m,s):
        # m主串，s模式串
        i,j = 0,0 
        pmt = self.pmt_table(s)
        while i<len(m) and j<len(s):
            # 模式串循环结束，匹配成功；否则失败
            if m[i] == s[j] or j== -1:
                i,j = i+1,j+1
            else:
                j = pmt[j]
        if j == len(s):
            return i-j
        return -1

In [374]:
t=Solution()
t.repeatedSubstringPattern("zzz") 

True

## [检测大写字母](https://leetcode-cn.com/problems/detect-capital/submissions/)

In [390]:
# 全部字母都是大写，比如"USA"。
# 单词中所有字母都不是大写，比如"leetcode"。
# 如果单词不只含有一个字母，只有首字母大写， 比如 "Google"。

class Solution:
    def detectCapitalUse(self, word: str) -> bool:
        num = 0
        for i in word:
            if i.isupper():
                num += 1
            
        word_len = len(word)
        return word_len ==  num or num == 0 or (num == 1 and word[0].isupper())

In [391]:
s = ['ABC','abc','Abc','aBC']
t=Solution()
for i in s:
    print(i,t.detectCapitalUse(i)) 

ABC True
abc True
Abc True
aBC False


## [最长特殊序列 Ⅰ](https://leetcode-cn.com/problems/longest-uncommon-subsequence-i/submissions/)

## [反转字符串 II](https://leetcode-cn.com/problems/reverse-string-ii/submissions/)

In [395]:
class Solution:
    def reverseStr(self, s: str, k: int) -> str:
        rest = ''
        return self.rest_reverse(rest,s,k)
        
    def rest_reverse(self,rest,s,k):
        
        if len(s)<k:
            rest += s[::-1]
            return rest
        else:
            part1 = s[:k]
            part2 = s[k:2*k]
            s = s[2*k:]
            rest += part1[::-1] + part2
            return self.rest_reverse(rest,s,k)
    
    
    def reverseStr_v2(self, s, k):
        a = list(s)
        for i in range(0, len(a), 2*k):
            a[i:i+k] = reversed(a[i:i+k])
        return "".join(a)

In [394]:
s = "abcdefg"
k = 2
   
t=Solution()
t.reverseStr(s,k)

'bacdfeg'

## [学生出勤记录 I](https://leetcode-cn.com/problems/student-attendance-record-i/submissions/)

In [412]:
class Solution:
    def checkRecord(self, s: str) -> bool:
        L_list = []
        A_num = 0
        index = 0
        for i in s:
            if i == 'A':
                A_num += 1
            if i ==  'L':
                if len(L_list) == 0 :
                    L_list.append(index)
                else:
                    if index - L_list[-1] == 1:
                        L_list.append(index)
                    else:
                        L_list = [index]
            if A_num>=2 or len(L_list)>=3:
                return False
            
            index += 1

        return A_num<2 and len(L_list)<3
               

In [413]:
s=["PPALLP","PPALLL"]
t=Solution()
for i in s:
    print(i,t.checkRecord(i))

PPALLP True
PPALLL False


## [计数二进制子串](https://leetcode-cn.com/problems/count-binary-substrings/submissions/)

In [523]:
class Solution:
    def countBinarySubstrings(self, s: str) -> int:
        s1,s2 = 0,1
        res = 0
        for i in range(1,len(s)):
            if s[i] == s[i-1]:
                s2 += 1
            else:
                res += min(s1,s2)
                s1 = s2
                s2 = 1
        res += min(s1,s2)
        return res
            
    def countBinarySubstrings_v2(self, s: str) -> int:
        str_map = {'0':'','1':''}
        p = None
        map_dict = {'0':'1','1':'0'}
        cum_dict = {'0':0,'1':0}
        res = []
        for i in s:            
            # cum的计算：入栈
            num_cum = cum_dict[i]
            reverse_cum = cum_dict[map_dict[i]]
            if reverse_cum == 0 or (num_cum != 0 and num_cum<reverse_cum):
                str_map[i] += i
                cum_dict[i] += 1 
            else:
                str_map[i] = i 
                cum_dict[i] = 1
            p = 0 if i == '0' else 1
           
            zero_cum,one_cum = cum_dict['0'],cum_dict['1']
            zero_str,one_str = str_map['0'],str_map['1']
            # 根据cum当前情况抽取符合情况的值
            
            if zero_cum != 0 and one_cum != 0:
                if zero_cum< one_cum:
                    strs = one_str[0:zero_cum] + zero_str
                    
                elif zero_cum > one_cum:
                    strs = zero_str[0:one_cum] + one_str
                else:
                    if p == 1:
                        strs = zero_str[0:one_cum] + one_str
                        cum_dict['0'] = 0
                    else:
                        strs = one_str[0:zero_cum] + zero_str
                        cum_dict['1'] = 0
                        
                        
                res.append(strs)
            
        return len(res)      

In [524]:
s="00110011"
t=Solution()
t.countBinarySubstrings(s)

6

## [转换成小写字母](https://leetcode-cn.com/problems/to-lower-case/submissions/)

## [旋转数字](https://leetcode-cn.com/problems/rotated-digits/submissions/)

In [538]:
class Solution:
    def rotatedDigits(self, N: int) -> int:
        match_str = '2569'
        not_match_str = '347'
        res = 0
        for i in range(2,N+1):
            s = str(i)
            res += all(char not in not_match_str for char in s) and any(char in match_str for char in s)
        return res
                    
                    
    def rotatedDigits_v2(self, N: int) -> int:
        map_dict ={'0':'0','1':'1','8':'8','2':'5','5':'2','6':'9','9':'6'}
        not_match_lst = '347'
        
        def rotatedN(N,num):
            if N<=1:
                return num 
            else:
                s = str(N)
                s1 = ''.join([map_dict.get(i,'') for i in s])
                if len(s1)<len(s) or s1 == s:
                    return rotatedN(N-1,num)
                else:
                    return rotatedN(N-1,num+1)
        
        return rotatedN(N,0)

In [539]:
s=10
t=Solution()
t.rotatedDigits(s)

4

## [唯一摩尔斯密码词](https://leetcode-cn.com/problems/unique-morse-code-words/)

In [553]:
class Solution:
    def uniqueMorseRepresentations(self, words: list) -> int:
        mos = [".-","-...","-.-.","-..",".","..-.","--.","....","..",".---","-.-",".-..","--","-.","---",".--.","--.-",".-.","...","-","..-","...-",".--","-..-","-.--","--.."]
        first = ord('a')
        map_dict = {}
        for i in range(26):
            map_dict.update({chr(first + i):mos[i]})
        s = set()
        for word in words:
            wordmos = ''.join([map_dict[i] for i in word])
            s.add(wordmos)
        return len(s)
    

In [554]:
words = ["gin", "zen", "gig", "msg"]
t=Solution()
t.uniqueMorseRepresentations(words)

2

## [最常见的单词](https://leetcode-cn.com/problems/most-common-word/submissions/)

In [603]:
class Solution:
    def mostCommonWord(self, paragraph: str, banned: list) -> str:
               
        i, unit_word,res =0,'',{}
        while i <len(paragraph):
            char = paragraph[i]
            if char.isalpha():
                unit_word += char.lower()
            elif unit_word:
                if unit_word not in banned:
                    res[unit_word] = res.get(unit_word,0) + 1
                
                unit_word = ''
    
            i += 1
        if unit_word != '':
            res[unit_word] = res.get(unit_word,0) + 1
        max_str,cnt = None,0
        for k in res:
            if res[k]>cnt:
                cnt = res[k]
                max_str = k
                
        return max_str

In [604]:
paragraph = "a, a, a, a, b,b,b,c, c"
banned = ["a"]
t=Solution()
t.mostCommonWord(paragraph,banned)

'b'

## [山羊拉丁文](https://leetcode-cn.com/problems/goat-latin/submissions/)

In [611]:
class Solution:
    def toGoatLatin(self, S: str) -> str:
        i, unit_word,res,index =0,'','',0
        para = 'aeiou'
        while i <len(S):
            char = S[i]
            if char.isalpha():
                unit_word += char
            elif unit_word:
                index += 1
                
                first = unit_word[0].lower()
                if first not in para:
                    unit_word =  unit_word[1:] + unit_word[0] 
                unit_word += 'ma'+ 'a'*index
                
                res += unit_word + char
                
                unit_word = ''
    
            i += 1
        
        index += 1
        first = unit_word[0].lower()
        if first not in para:
            unit_word =  unit_word[1:] + unit_word[0] 
        unit_word += 'ma'+ 'a'*indx

        res += unit_word
        return res   

In [612]:
S="I speak Goat Latin"
t=Solution()
t.toGoatLatin(S)

Latin


'Imaa peaksmaaa oatGmaaaa atinLmaaaa'

## [亲密字符串](https://leetcode-cn.com/problems/buddy-strings/)

In [625]:
class Solution:
    def buddyStrings(self, a: str, b: str) -> bool:
        if len(a) != len(b):
            return False
        if a == b:
            for i in range(len(a)-1):
                if a[i] in a[(i+1):]:
                    return True
            return False
        else:
            alist,blist = list(a),list(b)
            index = 0
            turn_index = 0
            for i in range(len(a)):
                if a[i] != b[i]:
                    index += 1
                    if index == 1:
                        turn_index = i
                    
                    if index == 2:
                        alist[turn_index],alist[i] = alist[i],alist[turn_index]
                        return alist == blist
            return False  

In [624]:
a='abcd'
b='cbad'
t=Solution()
t.buddyStrings(a, b)

True

## [特殊等价字符串组](https://leetcode-cn.com/problems/groups-of-special-equivalent-strings/submissions/)

In [672]:
class Solution:
    from itertools import combinations
    def numSpecialEquivGroups(self, A: list) -> int:
        s = set()
        for word in A:
            s.add(''.join(sorted(word[::2]) + sorted(word[1::2])))
        return len(s)   

In [674]:
A=["abcd","cdab","cbad","xyzz","zzxy","zzyx"]
# A=["abc","acb","bac","bca","cab","cba"]
t=Solution()
t.numSpecialEquivGroups(A)

3

## [长按键入](https://leetcode-cn.com/problems/long-pressed-name/)

In [408]:
class Solution:
    def isLongPressedName(self, name: str, typed: str) -> bool:
        if len(typed)<len(name):
            return False 
        i,j = 0,0 
        while i<len(name) and j<len(typed):
           
            c1 = name[i]
            c2 = typed[j]
            
            if c1 == c2:
                i,j = i+1,j+1
                
            else:
                if i-1>=0 and c2 == name[i-1]:
                    j += 1
                else:
                    return False
            
        if j<len(typed):
            return typed[j:] == name[-1]*(len(typed)-j)
        elif i >= len(name) and j == len(typed):
            return True
        else:
            return False

In [409]:
name = "pyplrz"
typed = "ppyypllr"
name = "alex"
typed="alexxr"
name="vtkgn"
typed="vttkgnn"
t=Solution()
t.isLongPressedName(name, typed)

True

## [独特的电子邮件地址](https://leetcode-cn.com/problems/unique-email-addresses/submissions/)

In [17]:
class Solution:
    def numUniqueEmails(self, emails: list) -> int:
        email_set = set()
        for email in emails:
            real_email = ''
            lens,i = len(email),0
            while i<lens:
                s = email[i]
                if s == '+' or s == '@':
                    while email[i] != '@':
                        i += 1
                    real_email += email[i:]
                    i = lens
                elif s != '.':
                    real_email += s
               
                i += 1
        
            email_set.add(real_email)
        return len(email_set)

In [18]:
emails = ["test.email+alex@leetcode.com","test.e.mail+bob.cathy@leetcode.com","testemail+david@lee.tcode.com"]
emails= ["test.email+alex@leetcode.com", "test.email@leetcode.com"]
t=Solution()
t.numUniqueEmails(emails)

1

 ## [字符串的最大公因子](https://leetcode-cn.com/problems/greatest-common-divisor-of-strings/)

In [55]:
class Solution:
    def gcdOfStrings(self, str1: str, str2: str) -> str:
        import math
        candidate_len = math.gcd(len(str1), len(str2))
        candidate = str1[: candidate_len]
        if str1 + str2 == str2 + str1:
            return candidate
        return ''

    def gcdOfStrings_v2(self, str1: str, str2: str) -> str:
        l1,l2 = len(str1),len(str2)
        if l1*l2 == 0:
            return ''
        gcd = self.gcd(l1,l2)        
        gcds = str1[0:gcd]
        strs = str1 + str2
        i,l = 0,len(strs)
        while i<l and strs[i:(i+gcd)] == gcds: 
                    i += gcd
  
        if i == l:
            return gcds
        else:
            return ''
                         
    def gcd(self,m,n):
        m,n = max(m,n),min(m,n)

        if m%n == 0:
            return n 
        else:
            diff = m - n 
            m,n = max(n,m-n),min(n,m-n)        
            return self.gcd(m,n)

In [56]:
str1 = "AA"
str2 = "A"
t=Solution()
t.gcdOfStrings(str1,str2)

'A'

## [IP 地址无效化](https://leetcode-cn.com/problems/defanging-an-ip-address/)

## [转变日期格式](https://leetcode-cn.com/problems/reformat-date/)

## [ “气球” 的最大数量](https://leetcode-cn.com/problems/maximum-number-of-balloons/submissions/)

In [60]:
class Solution:
    def maxNumberOfBalloons(self, text: str) -> int:
        s = "balloon"
        scnt = {}
        for i in s:
            if i not in scnt:
                scnt[i] = [1,0]
            else:
                scnt[i][0] += 1
        for j in text:
            if j in scnt:
                scnt[j][1] += 1
        num = None
        
        for k in scnt:
            num1 = scnt[k][1]//scnt[k][0]
            if num is None or num1<num:
                num = num1
        return num 

In [61]:
text = "loonbalxballpoon"
t=Solution()
t.maxNumberOfBalloons(text)

2

## [分割平衡字符串](https://leetcode-cn.com/problems/split-a-string-in-balanced-strings/solution/)

## [解码字母到整数映射](https://leetcode-cn.com/problems/decrypt-string-from-alphabet-to-integer-mapping/)

In [79]:
class Solution:
    def freqAlphabets(self, s: str) -> str:
        def get(st):
            return chr(int(st)+96)
        
        i,res = 0,''
        while i<len(s):
            if i+2<len(s) and s[i+2] == '#':
                res += get(s[i:i+2])
                i += 2
            else:
                res += get(s[i])
            i += 1
        return res 
   

In [80]:
t=Solution()
t.freqAlphabets("10#11#12")

'jkab'

## [删除回文子序列](https://leetcode-cn.com/problems/remove-palindromic-subsequences/submissions/)

## [上升下降字符串](https://leetcode-cn.com/problems/increasing-decreasing-string/submissions/)

In [102]:
class Solution:
    def sortString(self, s: str) -> str:
        ch = [0]*26
        for i in s:
            ch[ord(i)-ord('a')] += 1
        res = ''
        while len(res)<len(s):
            for i in range(26):
                if ch[i]:
                    res += chr(i + ord('a'))
                    ch[i] -= 1
            for i in range(25,-1,-1):
                if ch[i]:
                    res += chr(i + ord('a'))
                    ch[i] -= 1
        return res

In [103]:
s = "leetcode"
t=Solution()
t.sortString(s)
# 输出："cdelotee"

'cdelotee'

## [生成每种字符都是奇数个的字符串](https://leetcode-cn.com/problems/generate-a-string-with-characters-that-have-odd-counts/)

## [数组中的字符串匹配](https://leetcode-cn.com/problems/string-matching-in-an-array/submissions/)

## [重新格式化字符串](https://leetcode-cn.com/problems/reformat-the-string/submissions/)

In [112]:
class Solution:
    def reformat(self, s: str) -> str:
        s1,s2='',''
        for i in s:
            if 'a'<=i<='z':
                s1 += i 
            else:
                s2 += i 
        if abs(len(s1)-len(s2))>=2:
            return ''
        if len(s) == 2:
            return s[::-1]

        if len(s1)>len(s2):
            m = s1[::-1]
            n = s2[::-1]
        else:
            m = s2[::-1]
            n = s1[::-1]
        res = ''

        for i in range(len(m)):
            res += m[i]
            if i<len(n):
                res += n[i]
        return res 

In [115]:
s = "a0b1c2"
t=Solution()
t.reformat(s)

'2c1b0a'

## [ 分割字符串的最大得分](https://leetcode-cn.com/problems/maximum-score-after-splitting-a-string/submissions/)

## [ 连续字符](https://leetcode-cn.com/problems/consecutive-characters/solution/)

## [旅行终点站](https://leetcode-cn.com/problems/destination-city/solution/)

In [127]:
class Solution:
    def destCity(self, paths: list) -> str:
        l = []
        r = []
        for i in paths:
            if i[0] not in r:
                l.append(i[0])
            else:
                r.remove(i[0])
            if i[1] not in l:
                r.append(i[1])
            else:
                l.remove(i[1])
        return r[0]

In [128]:
s=[["London","New York"],["New York","Lima"],["Lima","Sao Paulo"]]
t=Solution()
t.destCity(s)

'Sao Paulo'

## [检查单词是否为句中其他单词的前缀](https://leetcode-cn.com/problems/check-if-a-word-occurs-as-a-prefix-of-any-word-in-a-sentence/)

## [ 千位分隔数](https://leetcode-cn.com/problems/thousand-separator/)

## [整理字符串](https://leetcode-cn.com/problems/make-the-string-great/)

## [替换所有的问号](https://leetcode-cn.com/problems/replace-all-s-to-avoid-consecutive-repeating-characters/)

In [191]:
class Solution:
    def modifyString(self, s: str) -> str:
        lens,lst,z_ord = len(s),list(s),ord('z')
        if lst[0] == '?':
            if lens == 1:
                return 'a'
            else:
                lst[0] = 'a' if lst[1] != 'a' else 'b'
        for i in range(1,len(s)-1):
            if lst[i] == '?':
                maxs = max(ord(lst[i-1]),ord(lst[i+1]))
                mins = min(ord(lst[i-1]),ord(lst[i+1]))
                
                if maxs + 1 <=z_ord:
                    lst[i] = chr(maxs+1)
                elif maxs - mins>1:
                    lst[i] = chr(maxs-1)
                else:
                    lst[i] = chr(mins-1)
        
        if lst[-1] == '?':
            if ord(lst[-2])+1<z_ord:
                lst[-1] = chr(ord(lst[-2])+1)
            else:
                lst[-1] = chr(ord(lst[-2])-1)
        return ''.join(lst)

In [192]:
s = "j?qg??b"
t=Solution()
t.modifyString(s)

'jrqghib'

## [重新排列单词间的空格](https://leetcode-cn.com/problems/rearrange-spaces-between-words/submissions/)

In [231]:
class Solution:
    def reorderSpaces(self, text: str) -> str:
        if len(text) == 1 and text !=' ':
            return text
        cnt,lst,word = 0,[],''
        for i in range(len(text)-1):
            
            char = text[i]
            if char == ' ':
                cnt += 1
            if char != ' ':
                word += char 
                if text[i+1] == ' ':
                    lst.append(word)
                    word = ''
                    
        if text[-1] == ' ':
            cnt += 1
        else:
            word += text[-1]
            lst.append(word)
        if len(lst) == 1:
            return ''.join(lst) + ' '*cnt 
        unit = cnt//(len(lst)-1)
        rest = cnt%(len(lst)-1)
        res = (' '*unit).join(lst) + ' '*rest
        return res      

In [232]:
text="  walks  udp package   into  bar a"
t=Solution()
t.reorderSpaces(text)

'walks  udp  package  into  bar  a '

## [括号的最大嵌套深度 ](https://leetcode-cn.com/problems/maximum-nesting-depth-of-the-parentheses/)

In [243]:
class Solution:
    def maxDepth(self, s: str) -> int:
        
        if len(s) == 0:
            return 0 
        j,cur_cnt = 0,0
        num = 0
        while j<len(s):
            i = s[j]
            
            if i == '(':
                cur_cnt += 1
                num = max(num,cur_cnt)
            elif i == ')':
                cur_cnt -= 1
            j += 1
        return num


In [244]:
s = "(1+(2*3)+((8)/4))+1"
t=Solution()
t.maxDepth(s)

3

## [两个相同字符之间的最长子字符串](https://leetcode-cn.com/problems/largest-substring-between-two-equal-characters/submissions/)

## [最大重复子字符串](https://leetcode-cn.com/problems/maximum-repeating-substring/submissions/)

## [统计一致字符串的数目](https://leetcode-cn.com/problems/count-the-number-of-consistent-strings/submissions/)

## [设计 Goal 解析器](https://leetcode-cn.com/problems/goal-parser-interpretation/submissions/)

## [重新格式化电话号码](https://leetcode-cn.com/problems/reformat-phone-number/solution/)

## [判断字符串的两半是否相似](https://leetcode-cn.com/problems/determine-if-string-halves-are-alike/submissions/)

## [替换隐藏数字得到的最晚时间](https://leetcode-cn.com/problems/latest-time-by-replacing-hidden-digits/submissions/)

## [最长的美好子字符串](https://leetcode-cn.com/problems/longest-nice-substring/)

## [交替合并字符串](https://leetcode-cn.com/problems/merge-strings-alternately/)

In [276]:
class Solution:
    def longestNiceSubstring(self, s: str) -> str:
        
        res = ''
        return self.findlonlyword(s,res)
                
  
    
    def findlonlyword(self,s,res):
        if len(s)<2:
            return res
        i = 0
        while i<len(s):
            p = s[i]
            p1 = p.upper()  if p.islower() else p.lower()
            if p1 in s:
                i+=1
            else:
                res1 = self.findlonlyword(s[:i],res)
                res2 = self.findlonlyword(s[i+1:],res)
                if len(res1)>=len(res2):
                    res = res1
                else:
                    res = res2
                return  res
        if i == len(s) and len(s)>len(res):
            res = s
        return res      

In [277]:
s ="dDzeE"
t=Solution()
t.longestNiceSubstring(s)

'dD'

## [字符串中第二大的数字](https://leetcode-cn.com/problems/second-largest-digit-in-a-string/submissions/)

## [仅执行一次字符串交换能否使两个字符串相等](https://leetcode-cn.com/problems/check-if-one-string-swap-can-make-strings-equal/)

## [字符串压缩](https://leetcode-cn.com/problems/compress-string-lcci/)

In [310]:
class Solution:
    def compressString(self, S: str) -> str:
        if not S:
            return ""
        ch = S[0]
        ans = ''
        cnt = 0
        for c in S:
            if c == ch:
                cnt += 1
            else:
                ans += ch + str(cnt)
                ch = c
                cnt = 1
        ans += ch + str(cnt)
        return ans if len(ans) < len(S) else S

In [311]:
S="bbbaac"
t=Solution()
t.compressString(S)

'bbbaac'

## [回文排列](https://leetcode-cn.com/problems/palindrome-permutation-lcci/submissions/)

In [327]:
class Solution:
    def canPermutePalindrome(self, s: str) -> bool:
        m = {}
        for i in s:
            m[i] = m.get(i,0)+ 1
        p = 0 
        
        for k in m:
            if m[k]%2 == 1:   
                p += 1
            if p==2:
                return False
        return p<2

In [328]:
s="tactcoa"
t=Solution()
t.canPermutePalindrome(s)

True

## [重新排列日志文件](https://leetcode-cn.com/problems/reorder-data-in-log-files/submissions/)

## [无重复字符的最长子串](https://leetcode-cn.com/problems/longest-substring-without-repeating-characters/)

In [456]:
class Solution:
    def lengthOfLongestSubstring(self, s: str) -> int:
        # 哈希集合是为了记录无重复的子字符串
        substr = set()
        n = len(s)
        # rk = -1 是相当于在字符串左指针的左侧
        rk,ans = -1,0 
        for i in range(len(s)):
            if i !=0:
                substr.remove(s[i-1])
            # 向右移动
            while rk+1<n and s[rk+1] not in substr:
                substr.add(s[rk+1])
                rk += 1
#             从i到rk是从i开始的最长无重复子字符串
            ans = max(ans,rk-i+1)
        return ans 
        
    def lengthOfLongestSubstring_v2(self, s: str) -> int:
        if len(s) <= 1:
            return len(s)
        num = 0
        res,his_lst ='',list(range(len(s)))
        while len(his_lst)>0:
            i = his_lst[0]
            
            while i<len(s):
                if i in his_lst:
                    his_lst.remove(i)

                while i<len(s) and s[i] not in res:
                    res += s[i]
                    i += 1

                num = max(num,len(res))
                res = ''
        return num  

In [457]:
s="bbcdef"
t=Solution()
t.lengthOfLongestSubstring(s)

5

## [Z 字形变换](https://leetcode-cn.com/problems/zigzag-conversion/submissions/)

In [602]:
class Solution:
    def convert(self, s: str, numRows: int) -> str:
        if numRows<2:
            return s 
        res = ['']*numRows
        i,direction = 0,-1
        for c in s:
            res[i] += c
            if i == 0 or i == numRows-1:
                direction = -direction
            i += direction
        return ''.join(res)
    
    
    def convert_v2(self, s: str, numRows: int) -> str:
        if numRows<=1:
            return s
        p,ni,i = 0,0,0
        lst = ['']*numRows
        while i<len(s):
            while i<len(s) and p==0 and ni<=numRows-1:
                lst[ni] += s[i]
                ni,i = ni+1,i+1
            ni,p= ni-1,1
            
            if p == 1 and ni == 0:
                i += 1
                
            while i<len(s) and p == 1 and ni>0:
                ni,i = ni-1,i+1          
                lst[ni] += s[i]
            ni,p = ni+1,0            
        return ''.join(res)

In [603]:
s = "PAYPALISHIRING"
# s='AB'
numRows = 3
t=Solution()
t.convert(s, numRows)

'PAHNAPLSIIGYIR'

## [字符串转换整数 (atoi)](https://leetcode-cn.com/problems/string-to-integer-atoi/submissions/)

In [679]:
INT_MAX = 2 ** 31 - 1
INT_MIN = -2 ** 31

class Automachine():
    def __init__(self):
        self.state = 'start'
        self.sign = 1
        self.ans = 0
        self.table = {'start':['start','signed','in_number','end'],
                     'signed':['end','end','in_number','end'],
                     'in_number':['end','end','in_number','end'],
                     'end':['end','end','end','end']}
    
    def get_colSeq(self,c):
        if c.isspace():
            return 0
        elif c == '-' or c=='+':
            return 1
        elif c.isdigit():
            return 2
        return 3
    
    def get(self,c):
        self.state = self.table[self.state][self.get_colSeq(c)]
        if self.state == 'in_number':
            self.ans = self.ans*10 + int(c)
            self.ans = min(self.ans,INT_MAX) if self.sign == 1 else min(self.ans,-INT_MIN)
        elif self.state == 'signed':
            self.sign = 1 if c == '+' else -1 
    
class Solution():
    def myAtoi(self, s: str) -> int:
        automachine = Automachine()
        for c in s:
            automachine.get(c)
        
        return automachine.ans*automachine.sign

In [680]:
class Solution:
    def myAtoi(self, s: str) -> int:
        if len(s) == 0:
            return 0
        left,right = - 2**31,2**31-1

        res = ''
        p = None
        while len(s)>=1 and s[0] == ' ':
            s = s[1:]
        if len(s)>=1 and s[0] in '-+':
            p = s[0]
            s = s[1:]
        else:
            p = '+'
            
        if len(s) == 0:
            return 0

        for i in s:
            if i == '0' and res == '':
                continue
            elif '0'<=i and i<='9':
                res += i 
            else:
                break 
        if res == '':
            return 0 
        if p == '-':
            num = max(-int(res),left)
        else:
            num = min(int(res),right)
                   
        return num

In [681]:
# s = "   +0 123"
s="-000000000000001"
s='-5-'
# s=" "
t=Solution()
t.myAtoi(s)
# 输出：-2147483648

-5

## [整数转罗马数字](https://leetcode-cn.com/problems/integer-to-roman/submissions/)

In [95]:
class Solution:
    
    def intToRoman(self, num: int) -> str:
        digits = [(1000, "M"), (900, "CM"), (500, "D"), (400, "CD"), (100, "C"), (90, "XC"), 
          (50, "L"), (40, "XL"), (10, "X"), (9, "IX"), (5, "V"), (4, "IV"), (1, "I")]
        map_lst = []
        for k,rom in digits:
            if num == 0:
                break 
            count,num = divmod(num,k)
            map_lst.append(rom*count)
        return ''.join(map_lst)
              

    def intToRoman_v2(self, num: int) -> str:
        map_dict = {1: 'I', 5: 'V', 10: 'X', 50: 'L', 100: 'C', 500: 'D',
                    1000: 'M', 4: 'IV', 9: 'IX', 40: 'XL', 90: 'XC', 400: 'CD', 900: 'CM'}
        unit_lst = [1,4,5,9,10,40,50,90,100,400,500,900,1000]
        i,index,div = -1,0,num
        res = ''
        while div>0:  
            div,mod = divmod(div,10)
            mod = mod*(10**index)
            index += 1   
            
            while i+1<len(unit_lst) and mod>=unit_lst[i+1]:
                i += 1
            unit = unit_lst[i]
            
            j = i
            while j>0 and (mod%unit)<unit_lst[j]:
                j -=  1
            pre_unit = 1 if j<1 else unit_lst[j]
            
            a,b = divmod(mod,unit)
            b = b//pre_unit

            res  = map_dict[unit]*a + map_dict[pre_unit]*b + res
        return res

In [96]:
t=Solution()
t.intToRoman(1984)

'MCMLXXXIV'

## [电话号码的字母组合](https://leetcode-cn.com/problems/letter-combinations-of-a-phone-number/submissions/)

In [118]:
map_dict = {'2':'abc','3':'def','4':'ghi','5':'jkl','6':'mno','7':'pqrs','8':'tuv','9':'wxyz'}
class Solution:
    def letterCombinations(self, digits: str) -> list:
        from functools import reduce
        if len(digits)<=1:
            return list(map_dict.get(digits,''))
        
        char_lst = [map_dict[i] for i in digits]
        return reduce(lambda x,y:self.product_twolst(x,y),char_lst)
        
    
    def product_twolst(self,lst1,lst2):
        from itertools import product
        return [''.join(i) for i in product(lst1,lst2)]

In [120]:
t=Solution()
s=t.letterCombinations('2345')

## [字符串相乘](https://leetcode-cn.com/problems/multiply-strings/)

In [121]:
class Solution:
    def multiply(self, num1: str, num2: str) -> str:
        n1,n2 = num1[::-1],num2[::-1]
        n1_lst = [int(n1[i])*(10**i) for i in range(len(n1))]
        n2_lst = [int(n2[i])*(10**i) for i in range(len(n2))]
        from itertools import product
        
        ans = 0
        for x,y in product(n1_lst,n2_lst):
            ans +=  x*y
        return ans 
           

In [123]:
t=Solution()
t.multiply('123','200')

24600

## [字母异位词分组](https://leetcode-cn.com/problems/group-anagrams/)

## [翻转字符串里的单词](https://leetcode-cn.com/problems/reverse-words-in-a-string/)

In [139]:
class Solution:
    def reverseWords(self, s: str) -> str:
        st = [i for i in s.split(' ') if ' ' not  in i and i !='']
        print(st)
        return ' '.join(st[::-1])

    def reverseWords_V2(self, s: str) -> str:
        # 使用双端队列
        left, right = 0, len(s) - 1
        # 去掉字符串开头的空白字符
        while left <= right and s[left] == ' ':
            left += 1
        
        # 去掉字符串末尾的空白字符
        while left <= right and s[right] == ' ':
            right -= 1
            
        d, word = collections.deque(), []
        # 将单词 push 到队列的头部
        while left <= right:
            if s[left] == ' ' and word:
                d.appendleft(''.join(word))
                word = []
            elif s[left] != ' ':
                word.append(s[left])
            left += 1
        d.appendleft(''.join(word))
        
        return ' '.join(d)

In [137]:
t=Solution()
t.reverseWords("  hello world  ")

['hello', 'world']


'world hello'

## [比较版本号](https://leetcode-cn.com/problems/compare-version-numbers/)

In [196]:
# class Solution:
#     def compareVersion(self, version1: str, version2: str) -> int:
#         s1 = [int(i) for i in version1.split('.')]
#         s2 = [int(i) for i in version2.split('.')]
#         i,j = 0,0
#         while i<len(s1) or j<len(s2):
#             x = s1[i] if i<len(s1) else 0
#             y = s2[j] if j<len(s2) else 0
#             if x > y:
#                 return 1
#             elif x<y:
#                 return -1 
            
#             i+= 1
#             j+= 1
#         return 0         

class Solution:
    def compareVersion(self, version1: str, version2: str) -> int:
        p1,p2 = 0,0
        l1,l2 = len(version1),len(version2)
        while p1<l1 or p2<l2:
            num1,p1 = self.getNextBlock(version1,l1,p1)
            num2,p2 = self.getNextBlock(version2,l2,p2)
            if num1 != num2:
                return 1 if num1>num2 else -1
        return 0
          
    def getNextBlock(self,version,length,p):
        if p>length-1:
            return 0,p
        end = p 
        while end<length and version[end] != '.':
            end += 1
        num = int(version[p:end]) if end != length else int(version[p:length])
        p = end + 1
        return num,p

In [197]:
version1 = "1.0.1"
version2 = "1"
t=Solution()
t.compareVersion(version1,version2)

1

## [基本计算器 II](https://leetcode-cn.com/problems/basic-calculator-ii/)

In [298]:
class Solution:
    def calculate(self, s: str) -> int:
        cal_lst = ['*','/','-','+']
        map_dict = {'+':1,'-':-1}
        
        last_sum,now_value = 0,0
        p,num,last_cal = 0,'',None
        
        while p<len(s):
            # 入栈
            while p<len(s) and s[p]>='0' and s[p]<='9':
                num += s[p]
                p += 1
                
            if num !='':
                num = int(num)
                if last_cal in cal_lst:
                    if last_cal == '-' or last_cal == '+':
                        last_sum += now_value
                        now_value = map_dict[last_cal]*num   
                    elif last_cal=='*' or last_cal == '/':
                        sign = -1 if now_value<0 else 1
                        now_value = now_value*num if last_cal == '*' else sign*(abs(now_value)//num)
                else:
                    last_sum += now_value
                    now_value = num 
                num = ''
            
            if p<len(s) and s[p] in cal_lst:
                last_cal = s[p]
                   
            p += 1
            
        last_sum += now_value 
        return  last_sum

In [299]:
s="14-3/2"
t=Solution()
t.calculate(s)

13

## [去除重复字母](https://leetcode-cn.com/problems/remove-duplicate-letters/)

In [315]:
class Solution():
    import collections
    def removeDuplicateLetters(self,s):
        res=[]
        # freq[c] 表示之后时间里c会出现的次数 初始化为每个字母的频数 遍历过程中减小
        freq=collections.Counter(s)
        for c in s:
            # c不在res中时再考虑是否添加
            if c not in res:
                # 添加前查看是否pop 注意这里是while 
                while len(res)>0 and c<res[-1] and freq[res[-1]]>0:
                    res.pop()
                res.append(c)
            # 无论是否添加c 它之后出现的频数都-1
            freq[c]-=1
        return ''.join(res)

In [316]:
s="bcabc"
t=Solution()
t.removeDuplicateLetters(s)

'abc'

## [压缩字符串](https://leetcode-cn.com/problems/string-compression/)

In [333]:
# class Solution(object):
#     def compress(self, chars):
#         res = []
#         ans = 1
#         for i in range(len(chars)-1):
#             s=chars[i]
#             s_next = chars[i+1]
#             if s == s_next:
#                 ans += 1
#             else:
#                 res.append(s)
#                 if ans>1:
#                     res.extend(str(ans))
#                 ans = 1
                
#         res.append(s)
#         if ans>1:
#             res.extend(str(ans))
#         chars = res 
#         return len(res)

class Solution:
    def compress(self, chars:list) -> int:
        stack = []
        chars.append('EOF')
        current = 0
        while current != 'EOF':
            current = chars.pop(0)
            stack.append(current)
            if len(set(stack)) > 1:
                next = stack.pop()
                chars.append(stack[0])
                if len(stack) > 1:
                    chars.extend(list(str(len(stack))))
                stack = [next]
        return len(chars)

In [334]:
chars=["a","a","b","b","c","c","c"]
t=Solution()
t.compress(chars)

6

## [验证IP地址](https://leetcode-cn.com/problems/validate-ip-address/)

In [424]:
class Solution:
    def validIPAddress(self, IP: str) -> str:
        if '.' not in IP and ':' in IP:
            return self.ipv6(IP)
        elif ':' not in IP and '.' in IP:
            return self.ipv4(IP)
        else:
            return "Neither"
        
    
    def ipv6(self,ip):
        i = 0
        cur_ans = ''
        index = 0
        while i<len(ip):
            cur = ip[i] 
            if cur.lower()<='z' and cur.lower() >'f':
                return "Neither"
            if cur != ':':
                cur_ans += cur 
            else:
                if len(cur_ans)>4 or len(cur_ans) == 0:
                    return "Neither"
                
                cur_ans = ''
                index += 1
            i += 1
        
        if index!=7:
            return "Neither"
        if len(cur_ans)>4 or len(cur_ans) == 0:
            return "Neither"
        return "IPv6"
        
        
    def ipv4(self,ip):
        i = 0
        cur_ans = ''
        index = 0
        while i<len(ip):
            cur = ip[i] 
            if cur>='0' and cur<='9':
                cur_ans += cur 
            elif cur == '.':                            
                if len(cur_ans) == 0  or not 0<=int(cur_ans)<=255 or len(str(int(cur_ans))) != len(cur_ans):
                    return "Neither"
                cur_ans = ''
                index += 1                
            else:
                return "Neither"
            i += 1
        if index!=3:
            return "Neither"
        if len(cur_ans) == 0  or not 0<=int(cur_ans)<=255 or len(str(int(cur_ans))) != len(cur_ans):
            return "Neither"
        
        return "IPv4"

In [423]:
IP ="1.0.1."
t=Solution()
t.validIPAddress(IP)

'Neither'

## [最长特殊序列 II](https://leetcode-cn.com/problems/longest-uncommon-subsequence-ii/)

In [512]:
class Solution:
    def findLUSlength(self, strs: list) -> int:
        ans = -1
        s=sorted(strs,key=len)
        i = 0 
        while len(s)>0:
            sub = s[0]
            rest = s[1:]
            j = 0
            while j<len(rest) and self.subStrs(sub,rest[j])==0:
                mark = self.subStrs(sub,rest[j])
                j += 1
            
            if j == len(rest):# 是独特子序列，将同等长度的字符串不需要判断，过滤掉
                ans = len(sub)
                s = [i for i in rest if len(i)>ans]
            else: #非独特子序列，将该元素以及相等元素过滤掉
                s = [i for i in rest if i!=sub]                   
        return ans 
    
    def subStrs(self,son,father):
        i,j = 0,0
        while i<len(son) and j<len(father):
            s1 = son[i]
            s2 = father[j]
            if s1 == s2:
                i = i+1
                j = j+1
            elif s1 != s2:
                j = j+1
        if i<len(son):# 非子序列
            return 0 
        else:
            return 1

In [513]:
strs=["aabbcc", "aabbcc","c","e"]
t=Solution()
t.findLUSlength(strs)

1

## [复数乘法](https://leetcode-cn.com/problems/complex-number-multiplication/)

In [595]:
class Solution:
    def complexNumberMultiply(self, a: str, b: str) -> str:
        a = self.complexNumber(a)
        b =self.complexNumber(b)
        res_x = a[0]*b[0]-a[1]*b[1]
        res_y = a[0]*b[1]+a[1]*b[0]
        union_sign = '+'
        return str(res_x) + union_sign + str(res_y) +'i'
    def complexNumber(self,comp):
        i = 0
        res = []
        ans = ''
        while i<len(comp):
            while i<len(comp) and comp[i] not in ['+','i']:
                ans += comp[i]
                i += 1
            else:
                if ans !='':
                    res.append(int(ans))
                ans = ''
            i += 1
        return res
    
#     def complexNumber(self,comp):
#         j = len(comp)-2
#         x,y = '',''
#         x_sign,y_sign = None,None
#         while j>-1:
#             while y_sign is None and comp[j]>='0' and comp[j]<='9':
#                 y = comp[j] + y 
#                 j -= 1
#             while j>-1 and comp[j] in ['+','-']:
#                 if y_sign!=-1 and x == '':
#                     y_sign = 1 if comp[j] =='+' else -1
#                 elif x != '':
#                     x_sign = 1 if comp[j] =='+' else -1
#                 j -= 1
                
#             while j>-1 and y_sign is not None and  comp[j]>='0' and comp[j]<='9':
#                 x  = comp[j] + x 
#                 j -= 1

#         x_sign = 1 if x_sign is None else x_sign
#         return (x_sign*int(x),y_sign*int(y))
    

In [597]:
a="3+-4i"
b="1+-1i"
t=Solution()
t.complexNumberMultiply(a, b)

'-1+-7i'

## [最小时间差](https://leetcode-cn.com/problems/minimum-time-difference/)

## [下一个更大元素 III](https://leetcode-cn.com/problems/next-greater-element-iii/)

In [642]:
class Solution:
    def nextGreaterElement(self, n: int) -> int:
        ns = str(n)
        j = len(ns)-1
        res = None
        while j>0:
            i= j-1
            while i>=0 and ns[i]>ns[j]:
                i -= 1
            if i != -1:
                ns_lst = list(ns)
                ns_lst[i],ns_lst[j] = ns_lst[j],ns_lst[i]
                l = ns_lst[:i+1]
                r = sorted(ns_lst[i+1:])
                new_n = int(''.join(l+r))
                if new_n>n and new_n<=2**31-1:
                    if res is None:
                        res = new_n
                    else:
                        res = min(res,new_n)
                
            j -= 1
        if res is None:
            return -1
        return res            
                

In [640]:
n=1234
t=Solution()
t.nextGreaterElement(n) 

1243

In [654]:
class Solution:
    def findDuplicate(self, paths: list) -> list:
        map_dict = dict()
        for s in paths:
            s = s.split(' ')
            path = s[0]
            s1=['/'.join([path,i]) for i in s[1:]]
            for i in s1:
                val = i.split('(')[0]
                key = i.split('(')[1].split(')')[0]
                map_dict[key] = map_dict.get(key,[]) + [val]
            res = [i for i in list(map_dict.values()) if len(i)>=2]
        return res
            

In [655]:
paths=["root/a 1.txt(abcd) 2.txt(efgh)", "root/c 3.txt(abcd)", "root/c/d 4.txt(efgh)", "root 4.txt(efgh)"]
t=Solution()
t.findDuplicate(paths)

[['root/a/1.txt', 'root/c/3.txt'],
 ['root/a/2.txt', 'root/c/d/4.txt', 'root/4.txt']]

## [重复叠加字符串匹配](https://leetcode-cn.com/problems/repeated-string-match/)

In [696]:
class Solution:
    def repeatedStringMatch(self, a: str, b: str) -> int:
        n = 0 
        while b not in a*n and (len(a)*n<=2*len(b) or n<2):
            n += 1
        if b in a*n:
            return n 
        else:
            return -1

In [660]:
a = "abcd"
b = "cdabcdab"
t=Solution()
t.repeatedStringMatch(a, b)

3

## [删除注释](https://leetcode-cn.com/problems/remove-comments/)

In [694]:
class Solution(object):
    def removeComments(self, source):
        in_block = False
        ans = []
        for line in source:
            i = 0
            if not in_block:
                newline = []
            while i < len(line):
                if line[i:i+2] == '/*' and not in_block:
                    in_block = True
                    i += 1
                elif line[i:i+2] == '*/' and in_block:
                    in_block = False
                    i += 1
                elif not in_block and line[i:i+2] == '//':
                    break
                elif not in_block:
                    newline.append(line[i])
                i += 1
            if newline and not in_block:
                ans.append("".join(newline))

        return ans

In [695]:
source = ["/*Test program */", "int main()", "{ ", "  // variable declaration ", "int a, b, c;", "/* This is a test", "   multiline  ", "   comment for ", "   testing */", "a = b + c;", "}"]
source=["struct Node{", "    /*/ declare members;/**/", "    int size;", "    /**/int val;", "};"]
t=Solution()
t.removeComments(source)

['struct Node{', '    ', '    int size;', '    int val;', '};']

## [自定义字符串排序](https://leetcode-cn.com/problems/custom-sort-string/)

In [733]:
class Solution:
    def customSortString(self, S: str, T: str) -> str:
        sort = list(S)
        ans = ['']*len(sort)
        notins = ''
        for i in T:
            if i in S:
                j = 0
                while i != sort[j]:
                    j += 1
                ans[j] += i 
            else:
                notins += i 
        res = ''.join(ans)+notins
        return res                 

In [732]:
S = "cebfa"
T = "efaaaabcd"
t=Solution()
t.customSortString(S,T)

'cebfaaaad'

## [情感丰富的文字](https://leetcode-cn.com/problems/expressive-words/)

In [772]:
class Solution:
    def expressiveWords(self, S: str, words: list) -> int:
        ans = 0 
        S_cnt = self.wordsCounts(S)
        for word in words:
            word_cnt = self.wordsCounts(word)
            ans += self.compare(S_cnt,word_cnt)
        return ans        
    
    def wordsCounts(self,s):
        import itertools
        return [(k, len(list(grp))) for k, grp in itertools.groupby(s)]
   
    def compare(self,S,l):
        if len(S)!=len(l):
            return 0
        i = 0 
        while i<len(S):
            s1,s2=S[i],l[i]
            if not (s1[0] == s2[0] and (s1[1] == s2[1] or (s1[1]>=3 and s1[1]>s2[1]))):       
                return 0 
            i += 1
        return 1

In [774]:
S="dddiiiinnssssssoooo"
words=["dinnssoo","ddinso","ddiinnso","ddiinnssoo","ddiinso","dinsoo","ddiinsso","dinssoo","dinso"]
# S="le"
# words=["lee"]
t = Solution()
t.expressiveWords(S,words)

3

In [771]:
import itertools
S="aabbbcccaaaaa"
[(k, len(list(grp))) for k, grp in itertools.groupby(S)]

[('a', 2), ('b', 3), ('c', 3), ('a', 5)]

## [隐藏个人信息](https://leetcode-cn.com/problems/masking-personal-information/)

In [798]:
class Solution:
    def maskPII(self, S: str) -> str:
        if '@' in S:
            return self.email_encode(S)
        else:
            return self.phoneNumberEncode(S)
        
    def email_encode(self,s):
        new_s = ''
        for i in s:
            if i.isupper():
                new_s += i.lower()
            else:
                new_s += i
                
        a,b = new_s.split('@')
        return '@'.join([a[0] + '*****' + a[-1],b])
    
    def phoneNumberEncode(self,s):
        new_s = ''
        for i in s:
            if '0'<=i<='9':
                new_s += i 
        basic_encode = '***-***-' + new_s[-4:] 
        if len(new_s)>10:
            return '+'+ '*'*(len(new_s)%10) + '-' + basic_encode
        else:
            return basic_encode
      

In [799]:
S="LeetCode@LeetCode.com"
S="86-(10)12345678"
t=Solution()
t.maskPII(S)

'+**-***-***-5678'

## [字符串中的查找与替换](https://leetcode-cn.com/problems/find-and-replace-in-string/)

In [30]:
class Solution:
    def findReplaceString(self, S: str, indexes: list, sources: list, targets: list) -> str:
        for i in range(len(indexes)):
            
            begin_index = indexes[i]
            source = sources[i]
            if S[begin_index:begin_index+len(source)] == source:
                p1 = S[:begin_index]
                p2 = targets[i]
                p3 = S[begin_index+len(source):]
                S = p1 + p2 + p3 
                                   
                indexes[i+1:] = [j + (len(p2)-len(source))*(j-indexes[i]>0) for j in indexes[i+1:]]
        return S      

In [31]:
S = "abcd"
indexes = [0,2]
sources = ["ab","ec"]
targets = ["eee","ffff"]

S="vmokgggqzp"
indexes =[3,5,1]
sources =["kg","ggq","mo"]
targets =["s","so","bfr"]

t=Solution()
t.findReplaceString(S, indexes, sources, targets)

'vbfrssozp'

## [字母移位](https://leetcode-cn.com/problems/shifting-letters/)

In [116]:
class Solution:
    def shiftingLetters(self, S: str, shifts: list) -> str:
        import numpy as np
        s2=np.cumsum(shifts[::-1])[::-1]
        s1 = [i%26 for i in s2]
        ans =''
        for i in range(len(s1)):
            shift_s = ord(S[i])+ s1[i]
            if shift_s>ord('z'):
                shift_s = shift_s-ord('z') + ord('a')-1
            ans += chr(shift_s)
        return ans

In [117]:
S = ""
shifts = [52]
S="ruu"
# "rul"
shifts =[26,9,17]
t=Solution()
t.shiftingLetters(S, shifts) 

'rul'

## [查找和替换模式](https://leetcode-cn.com/problems/find-and-replace-pattern/)

In [134]:
class Solution:
    def findAndReplacePattern(self, words: list, pattern: str) -> list:
        return list(filter(lambda x:self.matchWord(x,pattern),words))
    
    def matchWord(self,word,pattern):
        if len(word)!= len(pattern):
            return False
        w_list = list(word)
        map_dict = {}
        for i in range(len(word)):
            s = word[i]
            p = pattern[i]
            if p in map_dict and map_dict[p] != s or (p not in map_dict and s in map_dict.values()):
                return False
            if p not in map_dict:
                map_dict[p] = s
            w_list[i] = p
        return ''.join(w_list) == pattern

In [135]:
words = ["abc","deq","mee","aqq","dkd","ccc"]
pattern = "abb"
t=Solution()
t.findAndReplacePattern(words,pattern)

['mee', 'aqq']

## [单词子集](https://leetcode-cn.com/problems/word-subsets/)

In [154]:
import collections
class Solution:
    def wordSubsets(self, A: list, B: list) -> list:
        cnt_map_lst = [collections.Counter(i) for i in B]
        max_dict ={}
        for map_dict in cnt_map_lst:
            for j in map_dict:
                if j not in max_dict or (j in max_dict and max_dict[j]<map_dict[j]):
                    max_dict[j] = map_dict[j]
        
        ans = []
       
        for word in A:
            if self.submatch(word,max_dict):
                ans.append(word)
        return ans 
    def submatch(self,word,B_cnt_map):
        word_cnt = collections.Counter(word)
        return all([x in word_cnt and B_cnt_map[x]<=word_cnt[x]  for x in B_cnt_map])

In [155]:
A=["amazon","apple","facebook","google","leetcode"]
B=["lo","eo"]
t=Solution()
t.wordSubsets(A,B)

['google', 'leetcode']

## [检查替换后的词是否有效](https://leetcode-cn.com/problems/check-if-word-is-valid-after-substitutions/)

In [167]:
class Solution(object):
    def isValid(self, s):
        """
        :type s: str
        :rtype: bool
        """
        left_stack =[]
        for i in s:
            if i == 'c':
                if len(left_stack)<2 or ''.join(left_stack[-2:]) !='ab':
                    return False 
                else:
                    left_stack.pop()
                    left_stack.pop()
            else:
                left_stack.append(i)
        return len(left_stack) == 0

In [168]:
s = "abcabcababcc"
t=Solution()
t.isValid(s)

True

##  [驼峰式匹配](https://leetcode-cn.com/problems/camelcase-matching/)

In [209]:
class Solution:
    def camelMatch(self, queries: list, pattern: str) -> list:
        return [self.wordMatch(i,pattern) for i in queries]
    def wordMatch(self,word,pattern):
        i,j = 0,0 
        while j<len(pattern) and i<len(word):
            while i<len(word) and word[i].islower() and word[i]!=pattern[j]:
                i += 1
            if i<len(word) and word[i].isupper() and word[i]!=pattern[j]:
                break 
            if j<len(pattern) and i<len(word):
                i += 1
                j += 1
                
        if j<len(pattern):
            return False 
        if i<len(word):
            return all([i.islower() for i in word[i+1:]])
        return True

In [210]:
queries = ["FooBar","FooBarTest","FootBall","FrameBuffer","ForceFeedBack"]
pattern = "FoBaT"
queries =["CompetitiveProgramming","CounterPick","ControlPanel"]
pattern="CooP"
t=Solution()
t.camelMatch(queries,pattern)

[False, False, True]

True

## [比较字符串最小字母出现频次](https://leetcode-cn.com/problems/compare-strings-by-frequency-of-the-smallest-character/)

In [227]:
class Solution:
    def numSmallerByFrequency(self, queries: list, words: list) ->list:
        ans_lst = []
        q = []
        words_lst = []
        for i in queries:
            q.append(self.calculator(i))
        for j in words:
            words_lst.append(self.calculator(j))

        w = sorted(words_lst)
        i = 0 
        ans_lst = []
        while i<len(q):
            cnt1 = q[i]
            count = 0
            for j in range(len(w)):
                cnt2 = w[j] 
                if cnt2>cnt1:
                    break
                count += 1
            ans_lst.append(len(w)-count)   
            i += 1          
        return ans_lst

    
    def calculator(self,s):
        import collections
        cnt_dict = collections.Counter(s)
        word,cnt = '',0
        for i in cnt_dict:
            if word == '' or (i<word):
                word,cnt = i,cnt_dict[i]
        return cnt 

In [228]:
queries=["aabbabbb","abbbabaa","aabbbabaa","aabba","abb","a","ba","aa","ba","baabbbaaaa","babaa","bbbbabaa"]
# queries=["aabbbabaa"]
words=["b","aaaba","aaaabba","aa","aabaabab","aabbaaabbb","ababb","bbb","aabbbabb","aab","bbaaababba","baaaaa"]
t=Solution()
t.numSmallerByFrequency(queries, words)

[6, 5, 0, 6, 11, 11, 11, 8, 11, 0, 6, 6]

## [?构建回文串检测](https://leetcode-cn.com/problems/can-make-palindrome-from-substring/)

In [240]:
class Solution:
    def canMakePaliQueries(self, s: str, queries: list) -> list:
        ans = []
        for lst in queries:
            sub_s = s[lst[0]:lst[1]+1]
            k = lst[-1]
            sub_ans = self.wordReplace(sub_s,k)
            ans.append(sub_ans)
        return ans 
    def wordReplace(self,s,k):
        import collections
        t = collections.Counter(s)
        res = 0
        for i in t:
            r = t[i]%2
            if r == 1:
                res += 1
        return res - 2*k<=1

In [266]:
class Solution:
    def canMakePaliQueries(self, s: str, queries: list) -> list:
        dp = [0]
        cur = 0
        # dp中第i个数表示s中前i个字符中出现的字符的奇偶数，1表示对应的字符出现奇数次，0表示对应字符出现偶数次或者未出现
        for t in s:
            cur ^= 1 << (ord(t) - ord('a'))
            dp.append(cur)
           
        res = []
        for (x, y, k) in queries:
            temp = dp[y+1] ^ dp[x]  # 对前面x个中某些字符出现次数异或前面y个字符出现次数刚好表示从x到y中字符出现的奇偶数
            # 统计二进制中1出现的次数
            t = 0
            while temp != 0:
                if temp % 2 == 1:
                    t += 1
                temp = temp // 2
            # 对奇数字符个数/2，向下取整
            t = t // 2
            res.append(False) if t > k else res.append(True)
        return res

## [字符的最短距离](https://leetcode-cn.com/problems/shortest-distance-to-a-character/)

## [*增减字符串匹配](https://leetcode-cn.com/problems/di-string-match/)

## [验证外星语词典](https://leetcode-cn.com/problems/verifying-an-alien-dictionary/)

## [*句子中的有效单词数](https://leetcode-cn.com/problems/number-of-valid-words-in-a-sentence/)


链接：https://leetcode-cn.com/problems/number-of-valid-words-in-a-sentence/solution/python-zheng-ze-biao-da-shi-by-wushaobei-wgx9/

## [* Excel表列名称](https://leetcode-cn.com/problems/excel-sheet-column-title/)